# Dependencies

In [1]:
from sys import path
path.append('../src/')

import cv2
import pickle as pkl
from os.path import join, exists
from glob import glob
from sys import stdout

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from icao_dataset import ICAODataset
from utils import load
from data_structures import Point, Eye, Rect
from global_config import FOLDER_IMAGES, FOLDER_MRKS, FILE_DATASET, IMAGE_SIZE

In [2]:
PATH_TO_ROOT = ".."

FOLDER_IMAGES = join(PATH_TO_ROOT, FOLDER_IMAGES)
FOLDER_MRKS = join(PATH_TO_ROOT, FOLDER_MRKS)
FILE_DATASET = join(PATH_TO_ROOT, FILE_DATASET)

print(FOLDER_IMAGES)
print(FOLDER_MRKS)
print(FILE_DATASET)

..\data/cropped_faces/images/
..\data/cropped_faces/ground_truth/
..\data/cropped_faces_eyes/dataset_dlib.pkl


In [3]:
def load_graph_from_pb(pb_file):
    graph = tf.Graph()

    graph_def = tf.GraphDef()
    graph_def.ParseFromString(open(pb_file, 'rb').read())

    with graph.as_default():
        tf.import_graph_def(graph_def)
    return graph

# Preprocessing 

In [4]:
MODEL_FACE_DETECTOR = "../vsproject/FVCSubmission/resources/models/face_detector.pb"

face_detector = load_graph_from_pb(MODEL_FACE_DETECTOR)

In [5]:
def run_graph(graph, inputs_list, input_tensor_names, output_tensor_names):
    assert(len(inputs_list) == len(input_tensor_names))

    with tf.Session(graph=graph) as sess:
        input_tensors = [graph.get_tensor_by_name(name) for name in input_tensor_names]
        output_tensors = [graph.get_tensor_by_name(name) for name in output_tensor_names]
        feed_dict = {tensor:value for tensor, value in zip(input_tensors, inputs_list)}
        return sess.run(output_tensors, feed_dict=feed_dict)


def run_face_detector(bgr_image):
    img = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    img = img.astype(np.uint8)
    height, width = img.shape[:2]
    
    inputs_list = [np.expand_dims(img, axis=0)]
    input_tensor_names = ['import/image_tensor:0']
    output_tensor_names = ['import/detection_boxes:0', 'import/detection_scores:0']
    boxes, scores = run_graph(face_detector, inputs_list, input_tensor_names, output_tensor_names)

    best_rect = boxes.squeeze()[scores.squeeze().argmax()]
    best_rect = best_rect * [height, width, height, width]
    best_rect = best_rect.astype(np.int)
    y1, x1, y2, x2 = best_rect
    return Rect.from_tl_br_points((x1, y1), (x2, y2))


def update_eye_position(eye, offset):
    return Eye.from_left_and_right_corners(
        left_corner=eye.left_corner + offset,
        right_corner=eye.right_corner + offset,
    )


def normalize_eye_position(eye, width, height):
    p = Point(width, height)
    return Eye.from_left_and_right_corners(
        left_corner=eye.left_corner / p,
        right_corner=eye.right_corner / p,
    )


def preprocessing(bgr_image):
    face_rect = run_face_detector(bgr_image)
    w, h = face_rect.width, face_rect.height
    
    pad_rect = Rect(face_rect.x ,face_rect.y, w + int(w*1.5), h + int(h*1.5))
    pad_rect.height= pad_rect.width = max(pad_rect.height, pad_rect.width)

    center_pad = Point((pad_rect.x + pad_rect.width) // 2, (pad_rect.y + pad_rect.height) // 2)
    center_face = Point((face_rect.x + face_rect.width) // 2, (face_rect.y + face_rect.height) // 2)
    offset_center = Point(center_face.x - center_pad.x, center_face.y - center_pad.y)
    pad_rect += offset_center

    im_h, im_w = bgr_image.shape[:2]
    tl_x, tl_y = pad_rect.tl()
    br_x, br_y = pad_rect.br()
    left = abs(min(0, pad_rect.x))
    top = abs(min(0, pad_rect.y))
    right = 0 if br_x < im_w else br_x - im_w
    bottom = 0 if br_y < im_h else br_y - im_h

    offset = Point(-tl_x, -tl_y)
    
    im_res = cv2.copyMakeBorder(bgr_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    pad_rect.x += left
    pad_rect.y += top
    
    l, t = pad_rect.tl()
    r, b = pad_rect.br()
    return im_res[t:b, l:r], offset

# Data Loading 

#### New labels, wrong eyes

In [6]:
if exists(FILE_DATASET):
    print("Dataset found. Loading...")
    dataset = pkl.load(open(FILE_DATASET, "rb"))
else:
    print("Generating dataset...")
    dataset = ICAODataset()
    dataset.load(FOLDER_IMAGES, FOLDER_MRKS, output_size=IMAGE_SIZE)
    pkl.dump(dataset, open(FILE_DATASET, "wb"), protocol=4)

Generating dataset...
5731 of 5731
5731 of 5731


#### Old labels, right eyes

In [7]:
list_mrks_fvc = [mrk.filepath.replace("cropped_faces", "all") for mrk in dataset.mrks]
mrks_fvc = load.mrk_files_from_list_files(list_mrks_fvc)

5731 of 5731


# Update Ground-Truth

In [8]:
N_FILES = len(mrks_fvc)

for i, (file, mrk) in enumerate(zip(dataset.image_files, mrks_fvc)):
    print(f"{i + 1} of {N_FILES}", end="\r")
    stdout.flush()
    
    file = file.replace("cropped_faces", "all")
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    
    _, offset = preprocessing(img)
    dataset.mrks[i].left_eye = update_eye_position(mrk.left_eye, offset)
    dataset.mrks[i].right_eye = update_eye_position(mrk.right_eye, offset)

In [9]:
dataset.save(
    folder_images=join(PATH_TO_ROOT, "data/cropped_faces_eyes/images"), 
    folder_mrks=join(PATH_TO_ROOT, "data/cropped_faces_eyes/ground_truth")
)

In [11]:
pkl.dump(dataset, open(FILE_DATASET, "wb"), protocol=4)

In [12]:
print(FILE_DATASET)

..\data/cropped_faces_eyes/dataset_dlib.pkl


# Normalized Ground-Truth

In [6]:
if exists(FILE_DATASET):
    print("Dataset found. Loading...")
    dataset = pkl.load(open(FILE_DATASET, "rb"))
else:
    print("Generating dataset...")
    dataset = ICAODataset()
    dataset.load(FOLDER_IMAGES, FOLDER_MRKS, output_size=IMAGE_SIZE)
    pkl.dump(dataset, open(FILE_DATASET, "wb"))

Dataset found. Loading...


In [8]:
N_FILES = len(dataset.image_files)

for i, (file, _, mrk) in enumerate(dataset):
    print(f"{i + 1} of {N_FILES}", end="\r")
    stdout.flush()
    
    height, width, _ = cv2.imread(file, cv2.IMREAD_ANYCOLOR).shape
#     dataset.mrks[i].left_eye = normalize_eye_position(mrk.left_eye, width, height)
#     dataset.mrks[i].right_eye = normalize_eye_position(mrk.right_eye, width, height)
    new_left_eye = normalize_eye_position(mrk.left_eye, width, height)
    new_right_eye = normalize_eye_position(mrk.right_eye, width, height)
    
    print(mrk.left_eye)
    print(new_left_eye)
    break

left corner: (0.6043613707165109, 0.470404984423676) right_corner: (0.6542056074766355, 0.470404984423676)
left corner: (0.0009413728515833503, 0.0007327180442736386) right_corner: (0.0010190118496520803, 0.0007327180442736386)


In [9]:
height, width

(642, 642)

In [ ]:
FILE_NORMALIZED = FILE_DATASET.replace(".pkl", "_normalized.pkl")
pkl.dump(dataset, open(FILE_NORMALIZED, "wb"), protocol=4)

In [ ]:
print(FILE_NORMALIZED)